# Imports

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import torch
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
import numpy as np
import torch 
from torchvision import datasets, transforms
# from pytorchcv.model_provider import get_model as ptcv_get_model # model

import matplotlib.pyplot as plt

In [6]:
from loss_landscape.my_pyhessian import hessian, utils# Hessian computation
from loss_landscape.my_pyhessian.density_plot import get_esd_plot

In [7]:
from loss_landscape.plot_2D import plot_2d_contour

In [8]:
# import sys; sys.path.append("..")
from models.smooth_cross_entropy import mean_smooth_crossentropy
from models.wide_res_net import WideResNet
from models.transformer.model.transformer import Transformer 
from models.gcn import GCN

from DatasetClass.cifar import Cifar
from DatasetClass.mitbih import MitBih
from DatasetClass.TUD import GraphDataset

# WideResNet for Cifar

## Loss landscape plot

### With SGD

Generate surface file

In [ ]:
# Run plot_surface.py for trained model
# Command to be run from within loss_lanscape folder !!
python plot_surface.py --model WideResNet --dataset cifar10 --x=-1:1:2 --y=-1:1:2 --model_file ../to_plot/model_cifar_SGD.pt --dir_type weights --xnorm filter --xignore biasbn --ynorm filter --yignore biasbn --plot --percentage=0.3 --batch_size=128 --loss_name smooth_crossentropy

Generate plots

In [7]:
surf_file = 'to_plot/model_cifar_SGD.pt_weights_xignore=biasbn_xnorm=filter_yignore=biasbn_ynorm=filter.h5_[-1.0,1.0,5]x[-1.0,1.0,5].h5'

plot_2d_contour(surf_file, 'train_loss', 0.1, 10, 0.5, False)

------------------------------------------------------------------
plot_2d_contour
------------------------------------------------------------------
len(xcoordinates): 5   len(ycoordinates): 5
[[19.47184278  4.94447712  6.3274922  10.84011077 38.55445198]
 [ 6.52683822  2.74590339  3.3255251   3.25660324 12.8048792 ]
 [ 3.97482543  1.80651281  0.46783798  1.99040413  5.71666521]
 [ 4.86280387  2.75262686  1.93810351  2.42843889  6.8996155 ]
 [24.73039846  9.45652653  5.2411354   3.49049795 11.69184616]]


### With SAM

Generate surface file

In [ ]:
# Run plot_surface.py for trained model
# Command to be run from within loss_lanscape folder !!
python plot_surface.py --model WideResNet --dataset cifar10 --x=-1:1:2 --y=-1:1:2 --model_file ../to_plot/model_cifar_SAM_rho1.pt --dir_type weights --xnorm filter --xignore biasbn --ynorm filter --yignore biasbn --plot --percentage=0.3 --batch_size=128 --loss_name smooth_crossentropy

Generate plots

In [8]:
surf_file = 'to_plot/model_cifar_halfSAM_rho05.pt_weights_xignore=biasbn_xnorm=filter_yignore=biasbn_ynorm=filter.h5_[-1.0,1.0,5]x[-1.0,1.0,5].h5'

plot_2d_contour(surf_file, 'train_loss', 0.1, 10, 0.5, False)

------------------------------------------------------------------
plot_2d_contour
------------------------------------------------------------------
len(xcoordinates): 5   len(ycoordinates): 5
[[2.40649363 2.45352721 2.37080363 2.33494368 3.27218929]
 [2.11266517 1.72538848 1.35355867 1.86808758 2.42179902]
 [1.96148256 1.27078478 0.87594321 1.30209623 2.05305649]
 [2.82968777 1.60050254 1.09351784 1.46998333 1.97798065]
 [7.75504659 2.75616385 1.72724978 1.85543291 2.30363232]]


## Eigenvalues of hessian

Load trained model and dataset

In [9]:
model_name = 'WideResNet'

In [11]:
# get dataset 
dataset = Cifar(0.3, 128, 2)
trainloader, testloader = dataset.train, dataset.test

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


Extract batches of data for computation

In [ ]:
num_batches = 1

inputs = None
targets = None

for ind, (data, tar) in enumerate(trainloader):
    if inputs is None:
        inputs = data
        targets = tar
    elif inputs is not None and ind < num_batches:
        inputs = torch.cat((inputs, data), 0)
        targets = torch.cat((targets, tar))
    else:
        break

[print(inputs.size(), targets.size()) if targets is not None else print(inputs.size())]

# we use cuda to make the computation fast
# model = model.cuda()
# inputs, targets = inputs.cuda(), targets.cuda()

### With SGD

In [56]:
stored = torch.load('to_plot/model_cifar_SGD.pt', map_location=lambda storage, loc: storage)

model = WideResNet(8, 2, 0.0, in_channels=3, labels=10)

if 'state_dict' in stored.keys():
    model.load_state_dict(stored['state_dict'])
else:
    model.load_state_dict(stored)
model.eval()

criterion = mean_smooth_crossentropy

Create the hessian computation module and compute eigenvalue density

In [ ]:
eig_file_name = 'wideresnet_sgd'

In [65]:
try:
    density_eigen = np.load('Eigenvalues/'+eig_file_name+'_eigen.npy')
    density_weight = np.load('Eigenvalues/'+eig_file_name+'_weight.npy')
except:
    hessian_comp = hessian(model, criterion, data=(inputs, targets), cuda=False, model_name = model_name)
    density_eigen, density_weight = hessian_comp.density(iter=100, n_v=1)
    np.save('Eigenvalues/'+eig_file_name+'_eigen.npy', density_eigen)
    np.save('Eigenvalues/'+eig_file_name+'_weight.npy', density_weight)
    
get_esd_plot(density_eigen, density_weight,'plots/eig_'+eig_file_name+'.png')

### With SAM

In [56]:
stored = torch.load('to_plot/model_cifar_halfSAM_rho0.5.pt', map_location=lambda storage, loc: storage)

model = WideResNet(8, 2, 0.0, in_channels=3, labels=10)

if 'state_dict' in stored.keys():
    model.load_state_dict(stored['state_dict'])
else:
    model.load_state_dict(stored)
model.eval()

criterion = mean_smooth_crossentropy

Create the hessian computation module and compute eigenvalue density

In [ ]:
eig_file_name = 'wideresnet_sam'

In [65]:
try:
    density_eigen = np.load('Eigenvalues/'+eig_file_name+'_eigen.npy')
    density_weight = np.load('Eigenvalues/'+eig_file_name+'_weight.npy')
except:
    hessian_comp = hessian(model, criterion, data=(inputs, targets), cuda=False, model_name = model_name)
    density_eigen, density_weight = hessian_comp.density(iter=100, n_v=1)
    np.save('Eigenvalues/'+eig_file_name+'_eigen.npy', density_eigen)
    np.save('Eigenvalues/'+eig_file_name+'_weight.npy', density_weight)

get_esd_plot(density_eigen, density_weight,'plots/eig_'+eig_file_name+'.png')

# Transformer for Mit-Bih

## Loss landscape plot

### With SGD

Generate surface file

In [ ]:
# Run plot_surface.py for trained model
# Command to be run from within loss_lanscape folder !!
python plot_surface.py --model Transformer --dataset mitbih --x=-1:1:3 --y=-1:1:3 --model_file ../to_plot/model_mitbih_SGD.pt --dir_type weights --xnorm filter --xignore biasbn --ynorm filter --yignore biasbn --plot --batch_size=128 --loss_name smooth_crossentropy

Generate plots

In [19]:
surf_file = 'to_plot/model_mitbih_SGD.pt_weights_xignore=biasbn_xnorm=filter_yignore=biasbn_ynorm=filter.h5_[-1.0,1.0,3]x[-1.0,1.0,3].h5'
plot_2d_contour(surf_file, 'train_loss', 0.1, 10, 0.5, False)

------------------------------------------------------------------
plot_2d_contour
------------------------------------------------------------------
len(xcoordinates): 3   len(ycoordinates): 3
[[1.01853037 0.66109276 0.96737492]
 [0.86159587 0.4306891  0.736637  ]
 [1.23280096 0.60532123 0.98124242]]


### With SAM

Generate surface file

In [ ]:
# Run plot_surface.py for trained model
# Command to be run from within loss_lanscape folder !!
python plot_surface.py --model Transformer --dataset mitbih --x=-1:1:3 --y=-1:1:3 --model_file ../to_plot/model_mitbih_SAM_rho1.pt --dir_type weights --xnorm filter --xignore biasbn --ynorm filter --yignore biasbn --plot --batch_size=128 --loss_name smooth_crossentropy

Generate plots

In [ ]:
surf_file = 

plot_2d_contour(surf_file, 'train_loss', 0.1, 10, 0.5, False)

## Eigenvalues of hessian

Load trained model and dataset

In [1]:
model_name = 'Transformer'

In [10]:
# get dataset 
dataset = MitBih(128,2)
trainloader = dataset.train

In [18]:
num_batches = 1 # Only one batch (size 128)

inputs = None
targets = None

for ind, (inp,tar) in enumerate(trainloader):
    if inputs is None:
        inputs = inp
        targets = tar
    elif inputs is not None and ind < num_batches:
        inputs = torch.cat((inputs, inp), 0)
        targets = torch.cat((targets, tar))
    else:
        break

[print(inputs.size(), targets.size()) if targets is not None else print(inputs.size())]

# we use cuda to make the computation fast
# model = model.cuda()
# inputs, targets = inputs.cuda(), targets.cuda()

torch.Size([128, 187, 1]) torch.Size([128, 1])


[None]

### With SGD

In [15]:
stored = torch.load('to_plot/model_mitbih_SGD.pt', map_location=lambda storage, loc: storage)

model = Transformer(d_model=200, n_head=2, max_len=5000, seq_len=187, ffn_hidden=128, n_layers=1, drop_prob=0.1, details=False, device=device).to(device=device)

if 'state_dict' in stored.keys():
    model.load_state_dict(stored['state_dict'])
else:
    model.load_state_dict(stored)
model.eval()

criterion = mean_smooth_crossentropy

Create the hessian computation module and compute eigenvalue density

In [16]:
eig_file_name = 'transformer_sgd'

In [ ]:
try:
    density_eigen = np.load('Eigenvalues/'+eig_file_name+'_eigen.npy')
    density_weight = np.load('Eigenvalues/'+eig_file_name+'_weight.npy')

except:
    hessian_comp = hessian(model, criterion, data=(inputs, targets), cuda=False, model_name = model_name)
    density_eigen, density_weight = hessian_comp.density(iter=25, n_v=1)
    np.save('Eigenvalues/'+eig_file_name+'_eigen.npy', density_eigen)
    np.save('Eigenvalues/'+eig_file_name+'_weight.npy', density_weight)
    
get_esd_plot(density_eigen, density_weight,'plots/eig_'+eig_file_name+'.png')

### With SAM

In [56]:
stored = torch.load('to_plot/model_mitbih_SAM_rho????.pt', map_location=lambda storage, loc: storage)

model = Transformer(d_model=200, n_head=2, max_len=5000, seq_len=187, ffn_hidden=128, n_layers=1, drop_prob=0.1, details=False, device=device).to(device=device)

if 'state_dict' in stored.keys():
    model.load_state_dict(stored['state_dict'])
else:
    model.load_state_dict(stored)
model.eval()

criterion = mean_smooth_crossentropy

Create the hessian computation module and compute eigenvalue density

In [ ]:
eig_file_name = 'transformer_sam'

Plot eigenvalue density and save

In [65]:
try:
    density_eigen = np.load('Eigenvalues/'+eig_file_name+'_eigen.npy')
    density_weight = np.load('Eigenvalues/'+eig_file_name+'_weight.npy')
except:
    hessian_comp = hessian(model, criterion, data=(inputs, targets), cuda=False, model_name = model_name)  
    density_eigen, density_weight = hessian_comp.density(iter=100, n_v=1)
    np.save('Eigenvalues/'+eig_file_name+'_eigen.npy', density_eigen)
    np.save('Eigenvalues/'+eig_file_name+'_weight.npy', density_weight)
    
get_esd_plot(density_eigen, density_weight,'plots/eig_'+eig_file_name+'.png')

# Graph Convolutional Network for Mutagenicity

## Loss landscape plot

### With ADAM

Generate surface file

In [ ]:
# Run plot_surface.py for trained model
# Command to be run from within loss_lanscape folder !!
python plot_surface.py --model GCN --dataset Mutagenicity --x=-1:1:5 --y=-1:1:5 --model_file ../to_plot/model_gcn_ADAM.pt --dir_type weights --xnorm filter --xignore biasbn --ynorm filter --yignore biasbn --plot --batch_size=64 --loss_name smooth_crossentropy

Generate plots

In [20]:
surf_file = 'to_plot/model_gcn_ADAM.pt_weights_xignore=biasbn_xnorm=filter_yignore=biasbn_ynorm=filter.h5_[-1.0,1.0,5]x[-1.0,1.0,5].h5'
plot_2d_contour(surf_file, 'train_loss', 0.1, 10, 0.5, False)

------------------------------------------------------------------
plot_2d_contour
------------------------------------------------------------------
len(xcoordinates): 5   len(ycoordinates): 5
[[3.08615565 1.91437817 1.18252456 0.84519386 1.04570138]
 [1.46932542 0.78792375 0.45455039 0.33527473 0.36048952]
 [0.78654933 0.3918781  0.29017311 0.29505509 0.32700589]
 [0.45569074 0.29535779 0.30361739 0.33285248 0.34023479]
 [0.44712234 0.30643782 0.33153099 0.36096561 0.39565134]]


### With SAM

Generate surface file

In [ ]:
# Run plot_surface.py for trained model
# Command to be run from within loss_lanscape folder !!
python plot_surface.py --model GCN --dataset Mutagenicity --x=-1:1:2 --y=-1:1:2 --model_file ../to_plot/model_gcn_SAM_rho0.3.pt --dir_type weights --xnorm filter --xignore biasbn --ynorm filter --yignore biasbn --plot --batch_size=64 --loss_name smooth_crossentropy

Generate plots

In [21]:
surf_file = 'to_plot/model_gcn_SAM_rho0.3.pt_weights_xignore=biasbn_xnorm=filter_yignore=biasbn_ynorm=filter.h5_[-1.0,1.0,5]x[-1.0,1.0,5].h5'

plot_2d_contour(surf_file, 'train_loss', 0.1, 10, 0.5, False)

------------------------------------------------------------------
plot_2d_contour
------------------------------------------------------------------
len(xcoordinates): 5   len(ycoordinates): 5
[[0.37221444 0.36809254 0.36637071 0.36550125 0.37177667]
 [0.36776778 0.36647096 0.36590144 0.36509451 0.36471522]
 [0.36611596 0.36547902 0.36536816 0.36510125 0.36340114]
 [0.36520004 0.36565852 0.3654609  0.36441553 0.37480429]
 [0.36442244 0.36556324 0.36752364 0.37093782 0.42730191]]


## Eigenvalues of hessian

Load trained model and dataset

In [24]:
model_name = 'GCN'

In [25]:
# get dataset 
dataset = GraphDataset('Mutagenicity', 70, 64)
trainloader = dataset.train_loader

In [26]:
num_batches = 1 # CHOOSE MORE

inputs = None
targets = None

for ind, data in enumerate(trainloader):
    if inputs is None:
        inputs = data
    elif inputs is not None and ind < num_batches:
        inputs = torch.cat((inputs, data), 0)
    else:
        break

[print(inputs.size(), targets.size()) if targets is not None else print(inputs.size())]

# we use cuda to make the computation fast
# model = model.cuda()
# inputs, targets = inputs.cuda(), targets.cuda()

(2043, 2043)


[None]

### With SGD

In [27]:
model = GCN(64, dataset.dataset.num_node_features, dataset.dataset.num_classes).to(device)
stored = torch.load('to_plot/model_gcn_ADAM.pt', map_location=lambda storage, loc: storage)

if 'state_dict' in stored.keys():
    model.load_state_dict(stored['state_dict'])
else:
    model.load_state_dict(stored)
model.eval()

criterion = mean_smooth_crossentropy

Create the hessian computation module and compute eigenvalue density

In [28]:
eig_file_name = 'gcn_adam'

In [ ]:
try:
    density_eigen = np.load('Eigenvalues/'+eig_file_name+'_eigen.npy')
    density_weight = np.load('Eigenvalues/'+eig_file_name+'_weight.npy')
except:
    hessian_comp = hessian(model, criterion, data=(inputs, targets), cuda=False, model_name = model_name)
    density_eigen, density_weight = hessian_comp.density(iter=100, n_v=1)
    np.save('Eigenvalues/'+eig_file_name+'_eigen.npy', density_eigen)
    np.save('Eigenvalues/'+eig_file_name+'_weight.npy', density_weight)

get_esd_plot(density_eigen, density_weight,'plots/eig_'+eig_file_name+'.png')

### With SAM

In [99]:
stored = torch.load('to_plot/model_gcn_SAM_rho0.3.pt', map_location=lambda storage, loc: storage)
model = GCN(64, dataset.dataset.num_node_features, dataset.dataset.num_classes).to(device)

if 'state_dict' in stored.keys():
    model.load_state_dict(stored['state_dict'])
else:
    model.load_state_dict(stored)
model.eval()

criterion = mean_smooth_crossentropy

Create the hessian computation module and compute eigenvalue density

In [100]:
eig_file_name = 'gcn_sam'

In [102]:
try:
    density_eigen = np.load('Eigenvalues/'+eig_file_name+'_eigen.npy')
    density_weight = np.load('Eigenvalues/'+eig_file_name+'_weight.npy')
except:
    hessian_comp = hessian(model, criterion, data=(inputs, targets), cuda=False, model_name = model_name)
    density_eigen, density_weight = hessian_comp.density(iter=100, n_v=1)
    np.save('Eigenvalues/'+eig_file_name+'_eigen.npy', density_eigen)
    np.save('Eigenvalues/'+eig_file_name+'_weight.npy', density_weight)
    
get_esd_plot(density_eigen, density_weight,'plots/eig_'+eig_file_name+'.png')